# Optimization

**Business Analytics Assignment 3** 

Paulo Rijnberg

Jules Huisman

Lieske Trommelen

Bernard Wezeman

Myrthe Wouters

In [1]:
# Global imports
import pandas as pd
from pulp import *

#### Load Data

In [2]:
df_consultancy = pd.read_csv('../data/RUL_consultancy_predictions_A3.csv', sep=';', index_col='id')

In [3]:
df_prediction = pd.read_excel('../prediction/predictions.xlsx', index_col = 'id')

#### Function to define variables for an optimization task

Function takes as input:
- df: dataframe with our predicitons or consultancy predictions
- T: time horizon:
- task: 1, 2 or 3, refers to optimization task 1, optimization task 2 and optimization task 3 respectively

Returns all needed pre-calculated variables for optimisation task

In [4]:
def create_variables(df, T, task):
    def get_penalty(g, j, t):
        """
        Calculates penalty if team g starts working on engine j at day t
        """
        RUL = df.loc[j].values[0]
        c_per_day = c[j]
        nr_days_costs = t - RUL + mu[g][j] - 1 

        if nr_days_costs > 0:
            costs = nr_days_costs * c_per_day
            return costs
        else:
            return 0

    def get_unmaintained_penalty(j, T):
        """
        Calculates cost of engine j if the maintenance is not done during the planning horizon T 
        """
        RUL = df.loc[j].values[0]
        c_per_day = c[j]
        nr_days_costs = T - RUL

        if nr_days_costs > 0:
            costs = nr_days_costs * c_per_day
            return costs
        else: return 0

    # List with engine IDs
    M = list(df[df['RUL'] <= T].index)

    # Dictionary with team types
    G = {1: 'A', 2: 'A', 3: 'B', 4: 'B'}
    
    # Dictionary with cost per day for each engine if it is not working
    c = {i:5 for i in range(1,21)}
    c.update({i:7 for i in range(21,41)})
    c.update({i:9 for i in range(41,61)})
    c.update({i:5 for i in range(61,81)})
    c.update({i:3 for i in range(81,101)})

    # Dictionary with maintenance duration for each engine for teams of type A
    mu_a = {i:4 for i in range(1,26)}
    mu_a.update({i:6 for i in range(26,51)})
    mu_a.update({i:3 for i in range(51,76)})
    mu_a.update({i:5 for i in range(76,101)})
    
    # Dictionary with maintenance duration for each engine for teams of type B
    mu_b = {i:mu_a[i]+1 for i in range(1,34)}
    mu_b.update({i:mu_a[i]+2 for i in range(34,68)})
    mu_b.update({i:mu_a[i]+1 for i in range(68,101)})

    # Single look-up dictionary to find the maintenance duration for a team (g) and engine (j)
    mu = {g: {j: mu_a[j] if typ=='A' else mu_b[j] for j in M} for g, typ in G.items()}

    # Dictionary that holds the cost for all possible team, engine, day combinations
    P = {g:{j:{t: get_penalty(g,j,t) for t in range(1, T+1)} for j in M} for g in G}
    
    # Dictionary that holds the cost for an engine if is not maintained during the planning period
    Q = {j: get_unmaintained_penalty(j, T) for j in M}
    
    # Define maximum costs (case if no maintenance is performed during T)
    max_costs = sum(Q.values())
    
    # Savings if team g starts working on enigne j on day t
    S = {g:{j:{t: Q[j] - P[g][j][t] if Q[j] - P[g][j][t] > 0 else 0 for t in range(1, T+1)} for j in M} for g in G}
    
    if task>=2:
        # Define number of engines teams can work on during time horizon
        k = {g: 2 if G[g] == 'A' else 2 for g in G.keys()}
    else: 
        k = None

    if task==3:
        # Engine regions
        L = {j: 1 if j < 34 else 2 for j in M}

        # Team regions
        R = {g: 1 if g in [1,3] else 2 for g in G}
    else:
        L = None
        R = None
    
    return M, G, mu, P, Q, max_costs, S, k, L, R

#### Function to create MIP

Takes as input:
- task: 1, 2 or 3, refers to optimization task 1, optimization task 2 and optimization task 3 respectively
- T: time horizon
- M: engines that have RUL $\leq$ T
- G: teams
- mu: maintenance durations for all (team, engine) combinations
- S: Savings Savings if team g starts working on enigne j on day t of all (team, engine, day) combinations
- k: number of engines teams can work on during planning horizon
- L: engine regions
- R: team regions

In [5]:
def create_model(task, T, M, G, mu, S, k, L, R):
    """
    Creates model
    """
    
    # Create the model
    model = LpProblem(name='maintenance-schedule-optimization-2', sense=LpMaximize)

    team_engine_day = [(g, j, t) for g in G for j in M for t in range(1, T+1)]

    # Set LpVariable parameters
    # Natural constraints included
    X = LpVariable.dicts(name='start_day', indexs=team_engine_day, cat='Binary')

    # Add objective function
    model += lpSum(S[g][j][t] * X[(g, j, t)] for g in G for j in M for t in range(1, T+1))
    

    # Add constraints
    # Maintenance for each engine at most once during planning horizon
    for j in M:
        model += lpSum(X[(g, j, t)] for g in G for t in range(1, T+1)) <= 1

    # Maintenance must be completed within the planning period
    for j in M:
        model += lpSum(X[(g, j, t)] * (t + mu[g][j] - 1) for g in G for t in range(1, T+1)) <= T
    
    # Teams can only start on one engine a day
    for g in G:
         for t in range(1, T+1):
                model += lpSum(X[(g, j, t)] for j in M) <= 1
    
    # Teams can only work at one engine at a time          
    for g in G:
        for t_a in range(1, T+1):
            for j_a in M:
                model += len(M) * X[(g, j_a, t_a)] + lpSum(X[(g, j, t)] \
                        for t in range(t_a+1, min(t_a + mu[g][j_a],T)) for j in M) <= len(M)
    
    if task >= 2:
        for g in G:
            model += lpSum(X[(g, j, t)] for j in M for t in range(1, T+1)) <= k[g]
    
    if task == 3:
        for g in G:
            for j in M:
                model += lpSum(X[(g, j, t)] for t in range(1, T+1)) * L[j] == \
                lpSum(X[(g, j, t)] for t in range(1, T+1)) * R[g]
                    
    return model

#### Function to show results

In [6]:
def results_lp(model, max_costs, M, G, P, Q, mu):
    """
    Prints results of solved MIP
    """
    #Print the status of solving
    print("Status = %s" % LpStatus[model.status])
    # Print the value of the objective
    print("Objective = %f" % value(model.objective))
    
    data = {j: {'team': None,
                'team_type': None,
                'start_day': None,
                'end_day': None,
                'penalty_costs': Q[j]}
           for j in M}
    
    #Print the value of the variables when value > 0 
    for v in model.variables():
        if v.varValue != None and v.varValue > 0:
            print(v.name, "=", v.varValue)
            team, engine, day = [int(var) for var in v.name[len('start_day)('):-1].split(',_')]
            team_type = G[team]
            start_day = day
            end_day = day + mu[team][engine] - 1
            penalty_costs = P[team][engine][day]
            
            data[engine]['team'] = team
            data[engine]['team_type'] = team_type
            data[engine]['start_day'] = start_day
            data[engine]['end_day'] = end_day
            data[engine]['penalty_costs'] = penalty_costs
            
    print('')
    print("Total penalty costs = " , float(max_costs) - value(model.objective))
    
    df = pd.DataFrame.from_dict(data, orient='index').sort_values(['team', 'start_day'])
    df.index.name = 'engine'
    return df

In [7]:
def main(df,task,T):
    """
    Defines variables for optimization task and creates and solves MIP in PuLP.
    """
    M, G, mu, P, Q, max_costs, S, k, L, R = create_variables(df, T = T, task=task)
    model = create_model(task, T, M, G, mu, S, k, L, R)
    model.solve()
    
    return results_lp(model, max_costs, M, G, P, Q, mu)

## Optimization Task 1

### Part B

In [8]:
task1b = main(df_prediction, task=1, T=20)
task1b

Status = Optimal
Objective = 452.000000
start_day_(1,_20,_9) = 1.0
start_day_(1,_34,_3) = 1.0
start_day_(1,_36,_13) = 1.0
start_day_(2,_66,_10) = 1.0
start_day_(2,_68,_7) = 1.0
start_day_(2,_76,_2) = 1.0
start_day_(2,_82,_13) = 1.0
start_day_(3,_31,_2) = 1.0
start_day_(3,_42,_9) = 1.0
start_day_(4,_35,_7) = 1.0
start_day_(4,_81,_1) = 1.0

Total penalty costs =  0.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
34,1,A,3,8,0
20,1,A,9,12,0
36,1,A,13,18,0
76,2,A,2,6,0
68,2,A,7,9,0
66,2,A,10,12,0
82,2,A,13,17,0
31,3,B,2,8,0
42,3,B,9,16,0


### Part C

In [10]:
task1c = main(df_consultancy, task=1, T=20)
task1c

Status = Optimal
Objective = 561.000000
start_day_(1,_20,_7) = 1.0
start_day_(1,_34,_1) = 1.0
start_day_(1,_56,_14) = 1.0
start_day_(1,_68,_11) = 1.0
start_day_(2,_35,_1) = 1.0
start_day_(2,_42,_7) = 1.0
start_day_(2,_82,_13) = 1.0
start_day_(3,_31,_7) = 1.0
start_day_(3,_66,_14) = 1.0
start_day_(3,_81,_1) = 1.0
start_day_(4,_49,_7) = 1.0
start_day_(4,_76,_1) = 1.0

Total penalty costs =  33.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
34,1,A,1,6,0
20,1,A,7,10,0
68,1,A,11,13,0
56,1,A,14,16,0
35,2,A,1,6,0
42,2,A,7,12,0
82,2,A,13,17,18
81,3,B,1,6,0
31,3,B,7,13,0


## Optimization Task 2

### Part B

In [12]:
task2b = main(df_prediction, task=2, T=20)
task2b

Status = Optimal
Objective = 414.000000
start_day_(1,_31,_1) = 1.0
start_day_(1,_68,_7) = 1.0
start_day_(2,_20,_10) = 1.0
start_day_(2,_76,_2) = 1.0
start_day_(3,_34,_1) = 1.0
start_day_(3,_42,_9) = 1.0
start_day_(4,_35,_7) = 1.0
start_day_(4,_81,_1) = 1.0

Total penalty costs =  38.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
31,1.0,A,1.0,6.0,0
68,1.0,A,7.0,9.0,0
76,2.0,A,2.0,6.0,0
20,2.0,A,10.0,13.0,0
34,3.0,B,1.0,8.0,0
42,3.0,B,9.0,16.0,0
81,4.0,B,1.0,6.0,0
35,4.0,B,7.0,14.0,0
36,NaN,None,NaN,NaN,14


### Part C

In [14]:
task2c = main(df_consultancy, task=2, T=20)
task2c

Status = Optimal
Objective = 489.000000
start_day_(1,_34,_1) = 1.0
start_day_(1,_42,_7) = 1.0
start_day_(2,_20,_7) = 1.0
start_day_(2,_35,_1) = 1.0
start_day_(3,_49,_7) = 1.0
start_day_(3,_76,_1) = 1.0
start_day_(4,_31,_8) = 1.0
start_day_(4,_81,_1) = 1.0

Total penalty costs =  105.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
34,1.0,A,1.0,6.0,0
42,1.0,A,7.0,12.0,0
35,2.0,A,1.0,6.0,0
20,2.0,A,7.0,10.0,0
76,3.0,B,1.0,6.0,15
49,3.0,B,7.0,14.0,0
81,4.0,B,1.0,6.0,0
31,4.0,B,8.0,14.0,0
56,NaN,None,NaN,NaN,18


### Part D

In [16]:
task2d = main(df_consultancy,task=2,T=25)
task2d

Status = Optimal
Objective = 760.000000
start_day_(1,_49,_7) = 1.0
start_day_(1,_76,_1) = 1.0
start_day_(2,_34,_1) = 1.0
start_day_(2,_42,_7) = 1.0
start_day_(3,_20,_3) = 1.0
start_day_(3,_31,_8) = 1.0
start_day_(4,_35,_1) = 1.0
start_day_(4,_56,_10) = 1.0

Total penalty costs =  278.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
76,1.0,A,1.0,5.0,10
49,1.0,A,7.0,12.0,0
34,2.0,A,1.0,6.0,0
42,2.0,A,7.0,12.0,0
20,3.0,B,3.0,7.0,0
31,3.0,B,8.0,14.0,0
35,4.0,B,1.0,8.0,0
56,4.0,B,10.0,14.0,0
36,NaN,None,NaN,NaN,7


## Optimisation Task 3

### Part B

In [18]:
task3b = main(df_prediction, task=3, T=20)
task3b

Status = Optimal
Objective = 336.000000
start_day_(1,_20,_10) = 1.0
start_day_(1,_31,_1) = 1.0
start_day_(2,_34,_1) = 1.0
start_day_(2,_68,_7) = 1.0
start_day_(4,_35,_7) = 1.0
start_day_(4,_76,_1) = 1.0

Total penalty costs =  116.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
31,1.0,A,1.0,6.0,0
20,1.0,A,10.0,13.0,0
34,2.0,A,1.0,6.0,0
68,2.0,A,7.0,9.0,0
76,4.0,B,1.0,6.0,0
35,4.0,B,7.0,14.0,0
36,NaN,None,NaN,NaN,14
42,NaN,None,NaN,NaN,36
66,NaN,None,NaN,NaN,15


### Part C

In [20]:
task3c = main(df_consultancy, task=3, T=20)
task3c

Status = Optimal
Objective = 363.000000
start_day_(1,_31,_1) = 1.0
start_day_(2,_34,_2) = 1.0
start_day_(2,_42,_8) = 1.0
start_day_(3,_20,_3) = 1.0
start_day_(4,_49,_7) = 1.0
start_day_(4,_76,_1) = 1.0

Total penalty costs =  231.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
31,1.0,A,1.0,6.0,0
34,2.0,A,2.0,7.0,0
42,2.0,A,8.0,13.0,0
20,3.0,B,3.0,7.0,0
76,4.0,B,1.0,6.0,15
49,4.0,B,7.0,14.0,0
35,NaN,None,NaN,NaN,84
56,NaN,None,NaN,NaN,18
66,NaN,None,NaN,NaN,10


### Part D

In [22]:
task3d = main(df_consultancy, task=3, T=25)
task3d

Status = Optimal
Objective = 579.000000
start_day_(1,_20,_3) = 1.0
start_day_(2,_35,_1) = 1.0
start_day_(2,_42,_8) = 1.0
start_day_(3,_31,_6) = 1.0
start_day_(4,_34,_1) = 1.0
start_day_(4,_49,_9) = 1.0

Total penalty costs =  459.0


,team,team_type,start_day,end_day,penalty_costs
engine,,,,,
20,1.0,A,3.0,6.0,0
35,2.0,A,1.0,6.0,0
42,2.0,A,8.0,13.0,0
31,3.0,B,6.0,12.0,0
34,4.0,B,1.0,8.0,0
49,4.0,B,9.0,16.0,18
36,NaN,None,NaN,NaN,7
37,NaN,None,NaN,NaN,28
41,NaN,None,NaN,NaN,18
